In [ ]:
# Test new container on Gradient with Python: nvcr.io/nvidia/pytorch:21.09-py3
# See repo https://github.com/Paperspace/test-updated-containers for details
#
# Last updated: Oct 21st 2021

In [1]:
import platform
print(platform.python_version())

3.8.10


In [2]:
# Install gradient breaks PyTorch
#!pip3 install gradient

In [3]:
#import gradient
#gradient.version.version

In [4]:
pip --version

pip 21.2.4 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
print(np.__version__)

1.21.2


In [6]:
import torch
print(torch.__version__)

1.10.0a0+3fd9dcf


In [7]:
!nvidia-smi

Thu Oct 21 17:59:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      On   | 00000000:00:05.0 Off |                    0 |
| N/A   26C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import torch

In [9]:
torch.cuda.is_available()

True

In [10]:
torch.cuda.current_device()

0

In [11]:
torch.cuda.device(0)

In [12]:
torch.cuda.device_count()

1

In [13]:
print(torch.cuda.get_device_name(0))

'A100-SXM4-40GB'

In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [15]:
!pip3 install ipywidgets

     |████████████████████████████████| 121 kB 26.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 26.1 MB/s eta 0:00:01


In [16]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [17]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [18]:
batch_size = 64

In [19]:
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [20]:
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu" # This line points it to the GPU if available
print("Using {} device".format(device))                 # And we see if it picked it up

Using cuda device


In [22]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [23]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [25]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
# Accuracy should increase over 50%
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.315752  [    0/60000]
loss: 2.297227  [ 6400/60000]
loss: 2.282156  [12800/60000]
loss: 2.270546  [19200/60000]
loss: 2.249927  [25600/60000]
loss: 2.226770  [32000/60000]
loss: 2.225973  [38400/60000]
loss: 2.206316  [44800/60000]
loss: 2.209154  [51200/60000]
loss: 2.156451  [57600/60000]
Test Error: 
 Accuracy: 45.3%, Avg loss: 2.162088 

Epoch 2
-------------------------------
loss: 2.179507  [    0/60000]
loss: 2.163758  [ 6400/60000]
loss: 2.111086  [12800/60000]
loss: 2.119395  [19200/60000]
loss: 2.074906  [25600/60000]
loss: 2.011624  [32000/60000]
loss: 2.036020  [38400/60000]
loss: 1.972389  [44800/60000]
loss: 1.982884  [51200/60000]
loss: 1.886508  [57600/60000]
Test Error: 
 Accuracy: 56.3%, Avg loss: 1.898455 

Epoch 3
-------------------------------
loss: 1.938554  [    0/60000]
loss: 1.899994  [ 6400/60000]
loss: 1.791507  [12800/60000]
loss: 1.822657  [19200/60000]
loss: 1.717125  [25600/60000]
loss: 1.663517  [32000/600